In [ ]:
!pip install imageai
!pip install deeplabcut
!pip install tensorflow-object-detection-
!pip install imagehash
!pip install pillow
!pip install scikit-image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# set path to our training data pictures with different emotions as subfolders
# data_pictures_origin = '/content/drive/MyDrive/Coin Project 1/Pictures'
data_pictures = '/content/drive/MyDrive/Coin Project 1/Development_2/Pictures_Without_Duplicates'
data_pictures_cropped = '/content/drive/MyDrive/Coin Project 1/Development_2/Pictures_Cropped'
data_pictures_labeled = '/content/drive/MyDrive/Coin Project 1/Development_2/Pictures_Labeled'
data_pictures_labeled2 = '/content/drive/MyDrive/Coin Project 1/Development_2/Pictures_Labeled2'

In [ ]:
# Clean the Images
from imageai.Detection import ObjectDetection
import os
import cv2
import pandas as pd

# set model path to private google drive
model_path = '/content/drive/MyDrive/Coin Project 1/Develpment/DLC/resnet50_coco_best_v2.1.0.h5'

object_detector = ObjectDetection() 
object_detector.setModelTypeAsRetinaNet()
object_detector.setModelPath(model_path)
object_detector.loadModel()
custom_cat = object_detector.CustomObjects(cat=True)

In [ ]:
padding = 0.1
# change directory to pictures
os.chdir(data_pictures)

# for each emotion folder
for folder in os.listdir(data_pictures):

  print("Begin cropping images from folder ", folder)
  picture_counter = 0 # counter for number of pictures per emotion
  emotion_path = data_pictures + '/' + str(folder)
  # for each file inside the emotion folder
  for file_name in os.listdir(emotion_path):
    input_image_path = emotion_path + '/' + file_name
    output_image_path = data_pictures_cropped + '/' + str(folder) + '/' + str(folder) + '_' + str(picture_counter) + '.jpg'
    
    # detect our cat in picture
    detections = object_detector.detectObjectsFromImage(
        input_image=input_image_path,
        output_image_path=output_image_path,
        minimum_percentage_probability=10,
        custom_objects=custom_cat)
    
    if detections: # if cat got detected with enough probability
      image = cv2.imread(input_image_path)
      height, width, channels = image.shape

      cat = (sorted(detections, key=lambda cat: cat['percentage_probability'], reverse=True))[0]

      # Retrieve edge coordinates for the detected cat, padding for certainty
      x1 = max(0, int(cat['box_points'][0] * (1 - padding)))
      y1 = max(0, int(cat['box_points'][1] * (1 - padding)))
      x2 = min(width, int(cat['box_points'][2] * (1 + padding)))
      y2 = min(width, int(cat['box_points'][3] * (1 + padding)))

      image = image[y1:y2, x1:x2]
      if image.size:
        cv2.imwrite(output_image_path, image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
        picture_counter += 1
      else: print('Empty Image')
    else: print('No cat detected: ' + input_image_path)

In [ ]:
# analyze_time_lapse_frames 
import deeplabcut
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.pose_estimation_tensorflow.core import predict
import tensorflow as tf
import os
import cv2

#project_config = '/content/drive/MyDrive/Coin Project 1/Develpment/DLC/COINs-Cat-Emotions-Team-01-2022-04-21-20220421T063744Z-001.zip (Unzipped Files)/COINs-Cat-Emotions-Team-01-2022-04-21/config.yaml'
path_train_config_full_cat = '/content/drive/MyDrive/Coin Project 1/Develpment/DLC/COINs-Cat-Emotions-Team-01-2022-04-21-20220421T063744Z-001.zip (Unzipped Files)/COINs-Cat-Emotions-Team-01-2022-04-21/dlc-models/iteration-0/COINs-Cat-EmotionsApr21-trainset95shuffle1/train/pose_cfg.yaml'
#Todo
path_train_config_own_labels = '/content/drive/MyDrive/Coin Project 1/Development_2/DLC_New_Keypoints/Emotion_Recognition_in_Cats-Sofie.zip (Unzipped Files)/Emotion_Recognition_in_Cats-Sofie/dlc-models/iteration-0/Emotion_Recognition_in_CatsMay31-trainset80shuffle1/train/pose_cfg.yaml'

dlc_cfg_full_cat = load_config(str(path_train_config_full_cat))
dlc_cfg_own_labels = load_config(str(path_train_config_own_labels))
print(dlc_cfg_full_cat)
print()
print(dlc_cfg_own_labels)
print()

sess_full_cat, inputs_full_cat, outputs_full_cat = predict.setup_pose_prediction(dlc_cfg_full_cat)      # sess, inputs, outputs
sess_own_labels, inputs_own_labels, outputs_own_labels = predict.setup_pose_prediction(dlc_cfg_own_labels)      # sess, inputs, outputs

In [ ]:
def getEmotionFromPath(imagePath):
  if "Alert" in imagePath: return 0
  elif "Neutral" in imagePath: return 1
  elif "Relaxed" in imagePath: return 2
  elif "Anger" in imagePath: return 3
  elif "Fear" in imagePath: return 4
  elif "Anxious" in imagePath: return 5
  else: return -1

def getEmotionFromPathMerged(imagePath):
  if "Alert" in imagePath: return 0
  elif "Neutral" in imagePath: return 1
  elif "Relaxed" in imagePath: return 2
  elif "Anger_Fear" in imagePath: return 3
  else: return -1

In [ ]:
# Helper methods

# check if keypoint was found
def keypoint_found(point):
    if point == [-1,-1]: return False # no keypoint found 
    return True


# method to check a list of x and y coordinates
# the first one that has a x and y coordinate will be returned
def get_first_keypoint(pointList):
    for point in pointList:
        if keypoint_found(point): return point
    return None

# mirroring the image horizontally along the vertical axis
def mirror_image(point):
    if keypoint_found(point): return [resizeWidth - point[0], point[1]]
    else: return point

In [ ]:
'''
Cat's body parts: Numbering, Body Part, (Column for X Coordinate, Column for Y Coordinate)
- 0 - Nose (1,2)
- 1 - L_Eye (3,4)
- 2 - R_Eye (5,6)
- 3 - L_Ear (7,8)
- 4 - R_Ear (9,10)
- 5 - Throat (11,12)
- 6 - Withers (13,14)
- 7 - TailSet (15,16)
- 8 - L_F_Paw (17,18)
- 9 - R_F_Paw (19,20)
- 10 - L_F_Wrist (21,22)
- 11 - R_F_Wrist (23,24)
- 12 - L_F_Elbow (25,26)
- 13 - R_F_Elbow (27,28)
- 14 - L_B_Paw (29,30)
- 15 - R_B_Paw (31,32)
- 16 - L_B_Hock (33,34)
- 17 - R_B_Hock (35,36)
- 18 - L_B_Stiffle (37,38)
- 19 - R_B_Stiffle (39,40)
'''


def check_cat_direction_and_rename_columns(data, isFullCat):
  # create df from data
  df = pd.DataFrame(data)
  data_out = []

  if isFullCat:
    # define keypoints
    for index, row in df.iterrows():
      emotion = row[0]

      Nose = [row[1], row[2]]
      L_Eye = [row[3], row[4]]
      R_Eye = [row[5], row[6]]
      L_Ear = [row[7], row[8]]
      R_Ear = [row[9], row[10]]
      Throat = [row[11], row[12]]
      Withers = [row[13], row[14]]
      TailSet = [row[15], row[16]]
      L_F_Paw = [row[17], row[18]]
      R_F_Paw = [row[19], row[20]]
      L_F_Wrist = [row[21], row[22]]
      R_F_Wrist = [row[23], row[24]]
      L_F_Elbow  = [row[25], row[26]]
      R_F_Elbow  = [row[27], row[28]]
      L_B_Paw  = [row[29], row[30]]
      R_B_Paw  = [row[31], row[32]]
      L_B_Hock  = [row[33], row[34]]
      R_B_Hock  = [row[35], row[36]]
      L_B_Stiffle  = [row[37], row[38]]
      R_B_Stiffle  = [row[39], row[40]]

      # get first keypoint that is available in list
      back = get_first_keypoint([TailSet, L_B_Paw, R_B_Paw])
      front = get_first_keypoint([Nose, L_Eye, R_Eye, Throat])

      # if cat is looking into right direction, mirror image on vertical axis. Then cat is looking left
      if back is not None and front is not None:
        if front[0] > back[0]:
          Nose = mirror_image(Nose)
          L_Eye = mirror_image(L_Eye)
          R_Eye = mirror_image(R_Eye)
          L_Ear = mirror_image(L_Ear)
          R_Ear = mirror_image(R_Ear)
          Throat = mirror_image(Throat)
          Withers = mirror_image(Withers)
          TailSet = mirror_image(TailSet)
          L_F_Paw = mirror_image(L_F_Paw)
          R_F_Paw = mirror_image(R_F_Paw)
          L_F_Wrist = mirror_image(L_F_Wrist)
          R_F_Wrist = mirror_image(R_F_Wrist)
          L_F_Elbow  = mirror_image(L_F_Elbow)
          R_F_Elbow  = mirror_image(R_F_Elbow)
          L_B_Paw  = mirror_image(L_B_Paw)
          R_B_Paw  = mirror_image(R_B_Paw)
          L_B_Hock  = mirror_image(L_B_Hock)
          R_B_Hock  = mirror_image(R_B_Hock)
          L_B_Stiffle  = mirror_image(L_B_Stiffle)
          R_B_Stiffle  = mirror_image(R_B_Stiffle)

      data_out.append([emotion, Nose[0], Nose[1], L_Eye[0], L_Eye[1], R_Eye[0], R_Eye[1], L_Ear[0], L_Ear[1], R_Ear[0], R_Ear[1], Throat[0], Throat[1], Withers[0], Withers[1], TailSet[0], TailSet[1], L_F_Paw[0], L_F_Paw[1], R_F_Paw[0], R_F_Paw[1], L_F_Wrist[0], L_F_Wrist[1],
                      R_F_Wrist[0],R_F_Wrist[1], L_F_Elbow[0], L_F_Elbow[1], R_F_Elbow[0], R_F_Elbow[1], L_B_Paw[0], L_B_Paw[1], R_B_Paw[0], R_B_Paw[1], L_B_Hock[0], L_B_Hock[1], R_B_Hock[0], R_B_Hock[1], L_B_Stiffle[0], L_B_Stiffle[1], R_B_Stiffle[0], R_B_Stiffle[1]])

    return pd.DataFrame(data = data_out, columns = ['emotions', 'Nose_x','Nose_y','L_Eye_x','L_Eye_y','R_Eye_x','R_Eye_y','L_Ear_x','L_Ear_y','R_Ear_x','R_Ear_y','Throat_x','Throat_y','Withers_x','Withers_y','TailSet_x','TailSet_y','L_F_Paw_x','L_F_Paw_y','R_F_Paw_x','R_F_Paw_y','L_F_Wrist_x','L_F_Wrist_y','R_F_Wrist_x','R_F_Wrist_y','L_F_Elbow_x','L_F_Elbow_y','R_F_Elbow_x','R_F_Elbow_y','L_B_Paw_x','L_B_Paw_y','R_B_Paw_x','R_B_Paw_y','L_B_Hock_x','L_B_Hock_y','R_B_Hock_x','R_B_Hock_y','L_B_Stiffle_x','L_B_Stiffle_y','R_B_Stiffle_x','R_B_Stiffle_y'])

  else:
    for index, row in df.iterrows():
      emotion = row[0]

      back_middle = [row[1], row[2]]
      tail_tip = [row[3], row[4]]
      lip_upper = [row[5], row[6]]
      lip_lower = [row[7], row[8]]
      ear_tip_left = [row[9], row[10]]
      ear_base_left = [row[11], row[12]]
      ear_tip_right = [row[13], row[14]]
      ear_base_right = [row[15], row[16]]

      # get first keypoint that is available in list
      back = get_first_keypoint([tail_tip, back_middle])
      front = get_first_keypoint([lip_lower, lip_lower, ear_tip_left, ear_tip_right])

      # if cat is looking into right direction, mirror image on vertical axis. Then cat is looking left
      if back is not None and front is not None:
        if front[0] > back[0]:
          back_middle = mirror_image(back_middle)
          tail_tip = mirror_image(tail_tip)
          lip_upper = mirror_image(lip_upper)
          lip_lower = mirror_image(lip_lower)
          ear_tip_left = mirror_image(ear_tip_left)
          ear_base_left = mirror_image(ear_base_left)
          ear_tip_right = mirror_image(ear_tip_right)
          ear_base_right = mirror_image(ear_base_right)

      data_out.append([emotion, back_middle[0], back_middle[1], tail_tip[0], tail_tip[1], lip_upper[0], lip_upper[1], lip_lower[0], lip_lower[1], 
                       ear_tip_left[0], ear_tip_left[1], ear_base_left[0], ear_base_left[1], ear_tip_right[0], ear_tip_right[1], ear_base_right[0], ear_base_right[1]])

      '''
      - back_middle
      - tail_tip
      - lip_upper
      - lip_lower
      - ear_tip_left
      - ear_base_left
      - ear_tip_right
      - ear_base_right
      '''
    print(data_out)
    return pd.DataFrame(data = data_out, columns = ['emotion', 'back_middle_x', 'back_middle_y', 'tail_tip_x', 'tail_tip_y', 'lip_upper_x', 'lip_upper_y','lip_lower_x','lip_lower_y', 'ear_tip_left_x', 'ear_tip_left_y', 'ear_base_left_x', 'ear_base_left_y', 'ear_tip_right_x', 'ear_tip_right_y', 'ear_base_right_x', 'ear_base_right_y'])

In [ ]:
# resizing images before using deeplabcut
from skimage.io import imread
from skimage.transform import resize
from skimage.util import img_as_ubyte
import cv2

resizeWidth = 300
data = []

os.chdir(data_pictures_cropped)
for folder in sorted(os.listdir(data_pictures_cropped)):
  print(f'Starting with folder {folder}')
  for filename in sorted(os.listdir(folder)):

    if filename != '.DS_Store':
      pathImage = data_pictures_cropped + '/' + folder + '/' + filename
      image = imread(pathImage)
      height = image.shape[0]
      width = image.shape[1]
      # print(f'Before Width: {width}; Before Height: {height}')

      resizeHeight = int(round(height * resizeWidth / width))
      image = resize(image, (resizeHeight, resizeWidth), anti_aliasing=False)
      # print(f'After Width: {image.shape[0]}; After Height: {image.shape[1]}')
      frame = img_as_ubyte(image)

      # Todo: SET THE RIGHT MODEL HERE: EITHER own_labels or full_cat
      pose = predict.getpose(frame, dlc_cfg_own_labels, sess_own_labels, inputs_own_labels, outputs_own_labels)

      scaleFactor = width / resizeWidth

      image = cv2.imread(pathImage)
      coordinates = []
      counterPoint = 0

      # add emotion
      coordinates.append(getEmotionFromPath(pathImage))

      for keypoint in pose:
        if (keypoint[2] > 0.1):
          x = int(round(keypoint[0]*1))
          y = int(round(keypoint[1]*1))


          
          '''#Remember, try this!
          x = int(round(keypoint[0]*scaleFactor))
          y = int(round(keypoint[1]*scaleFactor))'''
          


          # Only if you want to label the pictures
          cv2.putText(image, str(counterPoint), (int(round(x*scaleFactor)),int(round(y*scaleFactor))), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

          coordinates.append(x)
          coordinates.append(y)
        else:
          coordinates.append(-1)
          coordinates.append(-1)
        counterPoint += 1
      
      # Only if you want to store the labeled pictures
      out_path_final = data_pictures_labeled2 + '/' + folder + '/' + filename
      cv2.imwrite(out_path_final, image) # write to labeled
      data.append(coordinates)

In [ ]:
from datetime import datetime
import pandas as pd
# check if we need to flip the picture because the cat is not looking to the left
# True = Full Cat, False = Own Key Points
df = check_cat_direction_and_rename_columns(data, False)
df.head(10)

In [ ]:
CSV_OUTPATH = '/content/drive/MyDrive/Coin Project 1/Development_2'
df.info()

filename = '/data_6_emotions_full_cat.csv'
filename_2 = '/new_key_points.csv'
filename_final = '/final_key_points.csv'

df.to_csv(CSV_OUTPATH + filename_2)

In [ ]:
full_cat_points = pd.read_csv(CSV_OUTPATH + filename).reset_index()
new_points = pd.read_csv(CSV_OUTPATH + filename_2).reset_index()
# Check if the emotions of the full_cat and the ones of the new_points are equal for all rows
is_equal = full_cat_points["emotions"].equals(new_points['emotion'])
print(is_equal)

In [ ]:
if is_equal:
  all_points = pd.concat([full_cat_points, new_points], axis=1)

In [ ]:
all_points.info()

In [ ]:
final_2.to_csv(CSV_OUTPATH + filename_final)